In [425]:
from ngsolve import *
from ngsolve.meshes import MakeStructured2DMesh
from netgen.geom2d import SplineGeometry, unit_square
from ngsolve.webgui import Draw
import time as timeit
from netgen.csg import *

*geometry and mesh*

In [426]:
n = 5
maxdofs = 3e5
c_low = 0
reduced = True; epsilon = 1e-8
mesh = MakeStructured2DMesh(quads=False, nx=n, ny=n)
#Draw(mesh)

**2D Stokes for Testing**

RT0 + Nedelec1_0 for velocity and velocity trace

In [427]:
# finite element spaces
V = MatrixValued(L2(mesh, order=0), mesh.dim, False)
W = HDiv(mesh, order=0, RT=True, dirichlet=".*")
M = HCurl(mesh, order=0, type1=True, dirichlet=".*")
Q = L2(mesh, order=0, lowest_order_wb=not reduced)
fes = V * W * M * Q
(L, u, uhat, p), (G, v, vhat, q) = fes.TnT()
gfu = GridFunction(fes)
Lh, uh, uhath, ph = gfu.components

n = specialcf.normal(mesh.dim)
h = specialcf.mesh_size

In [428]:
# bilinear and linear forms
def tang(v):
        return v - (v*n)*n

In [429]:
# exact solution
u_exact1 = x ** 2 * (x - 1) ** 2 * 2 * y * (1 - y) * (2 * y - 1)
u_exact2 = y ** 2 * (y - 1) ** 2 * 2 * x * (x - 1) * (2 * x - 1)
u_exact = CF((u_exact1, u_exact2))

L_exactXX = (2 * x * (x - 1) ** 2 * 2 * y * (1 - y) * (2 * y - 1)
                        + x ** 2 * 2 * (x - 1) * 2 * y * (1 - y) * (2 * y - 1))
L_exactXY = (x ** 2 * (x - 1) ** 2 * 2 * (1 - y) * (2 * y - 1)
                        - x ** 2 * (x - 1) ** 2 * 2 * y * (2 * y - 1)
                        + 2 * x ** 2 * (x - 1) ** 2 * 2 * y * (1 - y))
L_exactYX = (y ** 2 * (y - 1) ** 2 * 2 * (x - 1) * (2 * x - 1)
                        + y ** 2 * (y - 1) ** 2 * 2 * x * (2 * x - 1)
                        + 2 * y ** 2 * (y - 1) ** 2 * 2 * x * (x - 1))
L_exactYY = (2 * y * (y - 1) ** 2 * 2 * x * (x - 1) * (2 * x - 1)
                        + y ** 2 * 2 * (y - 1) * 2 * x * (x - 1) * (2 * x - 1))
L_exact = CF((L_exactXX, L_exactXY, L_exactYX, L_exactYY), dims=(2, 2))

p_exact = x * (1 - x) * (1 - y) - 1 / 12

Plain solver

In [430]:
a = BilinearForm(fes, symmetric=False, condense=True)
if reduced:
        a += epsilon * p * q * dx
a += (InnerProduct(L, G) + c_low*u*v - p*div(v) - div(u)*q) * dx
a += (-G*n * ((u*n)*n + tang(uhat)) + L*n * ((v*n)*n + tang(vhat))) * dx(element_boundary=True)

f = LinearForm(fes)
f += (-(4 * y * (1 - y) * (2 * y - 1) * ((1 - 2 * x) ** 2 - 2 * x * (1 - x))
                + 12 * x ** 2 * (1 - x) ** 2 * (1 - 2 * y))
        + (1 - 2 * x) * (1 - y)) * v[0] * dx
f += (-(4 * x * (1 - x) * (1 - 2 * x) * ((1 - 2 * y) ** 2 - 2 * y * (1 - y))
                + 12 * y ** 2 * (1 - y) ** 2 * (2 * x - 1))
        - x * (1 - x)) * v[1] * dx
f += c_low * u_exact * v * dx

def SolveBVP_plain(level, drawResult=False):
    t0 = timeit.time()
    fes.Update()
    gfu.Update()
    a.Assemble()
    f.Assemble()

    t1 = timeit.time()

    # dirichlet BC
    # homogeneous Dirichlet assumed
    f.vec.data += a.harmonic_extension_trans * f.vec
    inv = a.mat.Inverse(fes.FreeDofs(True), inverse='umfpack')
    gfu.vec.data += inv * f.vec
    gfu.vec.data += a.harmonic_extension * gfu.vec
    gfu.vec.data += a.inner_solve * f.vec

    
    t2 = timeit.time()

    print("Assemble: %.2e, Solve: %.2e" % (t1 - t0, t2 - t1))
    if drawResult:
        Draw(uh, mesh)

Now projection into CR space and solve

In [431]:
from ngsolve.krylovspace import CGSolver, MinResSolver, GMResSolver

V_cr = FESpace('nonconforming', mesh, dirichlet='.*')
Q_cr = L2(mesh, order=0, lowest_order_wb=not reduced)
fes_cr = V_cr * V_cr * Q_cr
(ux_cr, uy_cr, p_cr), (vx_cr, vy_cr, q_cr) = fes_cr.TnT()

u_cr = CF((ux_cr, uy_cr))
v_cr = CF((vx_cr, vy_cr))
GradU_cr = CF((grad(ux_cr), grad(uy_cr)))
GradV_cr = CF((grad(vx_cr), grad(vy_cr)))
divU_cr = grad(ux_cr)[0] + grad(uy_cr)[1]
divV_cr = grad(vx_cr)[0] + grad(vy_cr)[1]

a_cr = BilinearForm(fes_cr, condense=reduced)
if reduced:
    a_cr += epsilon * p_cr * q_cr * dx
# TODO: Mass matrix projection on RT!!!!
a_cr += (InnerProduct(GradU_cr, GradV_cr) + c_low * u_cr * v_cr \
         - p_cr * divV_cr - divU_cr * q_cr) * dx

f_cr = LinearForm(fes_cr)
f_cr += (-(4 * y * (1 - y) * (2 * y - 1) * ((1 - 2 * x) ** 2 - 2 * x * (1 - x))
                + 12 * x ** 2 * (1 - x) ** 2 * (1 - 2 * y))
        + (1 - 2 * x) * (1 - y)) * v_cr[0] * dx
f_cr += (-(4 * x * (1 - x) * (1 - 2 * x) * ((1 - 2 * y) ** 2 - 2 * y * (1 - y))
                + 12 * y ** 2 * (1 - y) ** 2 * (2 * x - 1))
        - x * (1 - x)) * v_cr[1] * dx
f_cr += c_low * u_exact * v_cr * dx

gfu_cr = GridFunction(fes_cr)
uhx_cr, uhy_cr, ph_cr = gfu_cr.components
uh_cr = CF((uhx_cr, uhy_cr))

# embedding from fes_cr to fes by L2 projection
mixmass = BilinearForm(trialspace=fes_cr, testspace=fes)
mixmass += (u_cr*n) * (v*n) * dx(element_boundary=True)
mixmass += tang(u_cr) * tang(vhat) * dx(element_boundary=True)

fesMass = BilinearForm(fes)
fesMass += (u*n) * (v*n) * dx(element_boundary=True)
fesMass += tang(uhat) * tang(vhat) * dx(element_boundary=True)

if not reduced:
    mixmass += p_cr * q * dx
    fesMass += p * q * dx

def SolveBVP_CR(level, drawResult=False):
    t0 = timeit.time()
    fes.Update(); fes_cr.Update()
    gfu.Update(); gfu_cr.Update()
    a.Assemble(); a_cr.Assemble()
    f.Assemble(); f_cr.Assemble()
    mixmass.Assemble(); fesMass.Assemble()

    udofs = BitArray(fes.ndof)
    udofs[:] = 0
    cur_ndof = V.ndof
    udofs[cur_ndof: cur_ndof+W.ndof] = W.FreeDofs(True)
    cur_ndof += W.ndof
    udofs[cur_ndof: cur_ndof+M.ndof] = M.FreeDofs(True)
    if not reduced:
        cur_ndof += M.ndof
        udofs[cur_ndof: cur_ndof+Q.ndof] = Q.FreeDofs(True)
    # global dofs mass mat => fesMass is diagonal
    fesM_inv = fesMass.mat.CreateSmoother(udofs)
    E = fesM_inv @ mixmass.mat # E: fes_cr => fes
    ET = mixmass.mat.T @ fesM_inv
    # direct solver for now.
    # TODO: MG solver for cr !!!!
    if reduced:
        inv_cr = a_cr.mat.Inverse(fes_cr.FreeDofs(True), inverse='umfpack')
    else:
        inv_cr = a_cr.mat.Inverse(fes_cr.FreeDofs(), inverse='umfpack')
    pre = E @ inv_cr @ ET
    t1 = timeit.time()

    # dirichlet BC
    # homogeneous Dirichlet assumed
    f.vec.data += a.harmonic_extension_trans * f.vec
    # gfu.vec.data += E @ inv_cr @ ET * f.vec
    if reduced: # SPD operator
        inv_fes = CGSolver(a.mat, pre, printrates=True, tol=1e-8, maxiter=30)
    else: # symmetric saddle point operator
        # TODO: FIX ME!!! MINRES SOLVER???
        inv_fes = GMResSolver(a.mat, pre, printrates=True, tol=1e-8, maxiter=30)
    gfu.vec.data += inv_fes * f.vec
    gfu.vec.data += a.harmonic_extension * gfu.vec
    gfu.vec.data += a.inner_solve * f.vec
        
    
    # gfu_cr.vec.data += a_cr.mat.Inverse(fes_cr.FreeDofs(True), inverse='umfpack') * f_cr.vec
    t2 = timeit.time()

    print("Assemble: %.2e, Solve: %.2e" % (t1 - t0, t2 - t1))
    if drawResult:
        Draw(uh, mesh)
        # Draw(uh_cr, mesh)

Projection into HDG-P0(?)

In [432]:
from ngsolve.la import EigenValues_Preconditioner
from prol import *
from mymg import *

W0 = VectorL2(mesh, order=1)
M0 = FacetFESpace(mesh, dirichlet=".*")
fes0 = M0 * M0 * Q * V * W0
(uhatx0, uhaty0, p0, L0, u0), (vhatx0, vhaty0, q0, G0, v0) = fes0.TnT()
uhat0 = CF((uhatx0, uhaty0))
vhat0 = CF((vhatx0, vhaty0))

gfu0 = GridFunction(fes0)
uhathx0, uhathy0, ph0, Lh0, uh0 = gfu0.components
uhath0 = CF((uhathx0, uhathy0))

et = meshTopology(mesh, mesh.dim)
et.Update()
prolM0 = FacetProlongationTrig(mesh, et)  # Facet prol
prolP0 = ElementProlongation0(mesh, et)  # constant pressure prol

alpha = 1
a0 = BilinearForm(fes0, symmetric=False, condense=True)
if reduced:
    a0 += epsilon * p0 * q0 * dx
a0 += (InnerProduct(L0, G0)) * dx
a0 += (-uhat0 * (G0 * n) + (L0 * n) * vhat0
          - uhat0 * n * q0 - p0 * vhat0 * n
          + alpha / h * (u0 - uhat0) * (v0 - vhat0)) * dx(element_boundary=True)
f0 = LinearForm(fes0)
f0 += (-(4 * y * (1 - y) * (2 * y - 1) * ((1 - 2 * x) ** 2 - 2 * x * (1 - x))
                + 12 * x ** 2 * (1 - x) ** 2 * (1 - 2 * y))
        + (1 - 2 * x) * (1 - y)) * v0[0] * dx
f0 += (-(4 * x * (1 - x) * (1 - 2 * x) * ((1 - 2 * y) ** 2 - 2 * y * (1 - y))
                + 12 * y ** 2 * (1 - y) ** 2 * (2 * x - 1))
        - x * (1 - x)) * v0[1] * dx
f0 += c_low * u_exact * v0 * dx

a0.Assemble()
pre0 = MultiGrid(a0.mat, prolM0, nc=M0.ndof,
                    coarsedofs=fes0.FreeDofs(True), w1=0.8,
                    nsmooth=4, sm="gs", var=True,
                    he=True, dim=2, wcycle=False)


def SolveBVP_HDGP0(level, drawResult):
    t0 = timeit.time()
    fes.Update(); fes0.Update()
    gfu.Update(); gfu0.Update()
    a.Assemble(); a0.Assemble()
    f.Assemble(); f0.Assemble()

    t1 = timeit.time()

    ## Update MG
    if level > 0:
        et.Update()
        pp = [fes0.FreeDofs(True)]
        pp.append(M0.ndof)
        pdofs = BitArray(fes0.ndof)
        pdofs[:] = 0
        inner = prolM0.GetInnerDofs(level)
        for j in range(mesh.dim):
            pdofs[j * M0.ndof:(j + 1) * M0.ndof] = inner
        # he_prol
        pp.append(a0.mat.Inverse(pdofs, inverse="sparsecholesky"))
        # bk smoother
        bjac = et.CreateSmoother(a0, {"blocktype": "vertexpatch"})
        pp.append(bjac)
        pre0.Update(a0.mat, pp)
    t2 = timeit.time()
    # estimate condition number
    # lams = np.array([1, 1])
    # inv = a0.mat.Inverse(fes0.FreeDofs(True), inverse='umfpack')
    lams = EigenValues_Preconditioner(mat=a0.mat, pre=pre0)
    inv = CGSolver(a0.mat, pre0, printrates=False, tol=1e-8, maxiter=100)
    t21 = timeit.time()
    # dirichlet BC
    # homogeneous Dirichlet assumed
    f0.vec.data += a0.harmonic_extension_trans * f0.vec
    # dirichlet BC
    # gfu0.vec.data += inv * f0.vec
    # it = inv.iterations
    gfu0.vec.data += a0.mat.Inverse(fes0.FreeDofs(True), inverse='umfpack') * f0.vec
    it = 1
    gfu0.vec.data += a0.harmonic_extension * gfu0.vec
    gfu0.vec.data += a0.inner_solve * f0.vec
    t3 = timeit.time()

    print(f"Time to find EIG Val: {t21 - t2:.1E}, MAX PREC LAM: {max(lams):.1E}; MIN PREC LAM: {min(lams):.1E}")
    print("IT: %2.0i" % (it), "cond: %.2e" % (max(lams) / min(lams)),
          "Assemble: %.2e Prec: %.2e Solve: %.2e" % (t1 - t0, t2 - t1, t3 - t21))
    if drawResult:
        Draw(uh0, mesh)
        # Draw(uh_cr, mesh)

Convergence rate test, 1st order convergence expected for Lh, uh and ph

In [433]:
drawResult = True
SolveBVP = SolveBVP_HDGP0

print(f'===== Reduced: {reduced} =====')
SolveBVP(0, drawResult)
print(f'level: {0}')
L2_uErr = sqrt(Integrate((uh - u_exact) * (uh - u_exact), mesh))
L2_LErr = sqrt(Integrate(InnerProduct((Lh - L_exact), (Lh - L_exact)), mesh))
L2_divErr = sqrt(Integrate(div(uh) * div(uh), mesh))
print(f"uh L2-error: {L2_uErr:.3E}")
print(f"Lh L2-error: {L2_LErr:.3E}")
print(f'uh divErr: {L2_divErr:.1E}')
print('==============================')
prev_uErr = L2_uErr
prev_LErr = L2_LErr
u_rate, L_rate = 0, 0
level = 1
while True:
    with TaskManager():
        mesh.ngmesh.Refine()
        # exit if total global dofs exceed a tol
        M.Update()
        if (M.ndof * mesh.dim > maxdofs):
            print(M.ndof * mesh.dim)
            break
        print(f'level: {level}')
        SolveBVP(level, drawResult)
        # ===== convergence check =====
        meshRate = 2
        L2_uErr = sqrt(Integrate((uh - u_exact) * (uh - u_exact), mesh))
        L2_LErr = sqrt(Integrate(InnerProduct((Lh - L_exact), (Lh - L_exact)), mesh))
        L2_divErr = sqrt(Integrate(div(uh) * div(uh), mesh))
        u_rate = log(prev_uErr / L2_uErr) / log(meshRate)
        L_rate = log(prev_LErr / L2_LErr) / log(meshRate)
        print(f"uh L2-error: {L2_uErr:.3E}, uh conv rate: {u_rate:.2E}")
        print(f"Lh L2-error: {L2_LErr:.3E}, Lh conv rate: {L_rate:.2E}")
        print(f'uh divErr: {L2_divErr:.1E}')
        print('==============================')
        prev_uErr = L2_uErr
        prev_LErr = L2_LErr
        level += 1

===== Reduced: True =====
Time to find EIG Val: 5.2E-04, MAX PREC LAM: 1.0E+00; MIN PREC LAM: 1.0E+00
IT:  1 cond: 1.00e+00 Assemble: 1.09e-02 Prec: 4.77e-07 Solve: 1.03e-03


WebGuiWidget(value={'ngsolve_version': '6.2.2105-289-g53df468f0', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, '…

level: 0
uh L2-error: 7.776E-03
Lh L2-error: 5.714E-02
uh divErr: 0.0E+00
level: 1
Time to find EIG Val: 1.0E-02, MAX PREC LAM: 1.4E+00; MIN PREC LAM: 9.9E-01
IT:  1 cond: 1.43e+00 Assemble: 1.46e-02 Prec: 2.86e-02 Solve: 1.99e-02


WebGuiWidget(value={'ngsolve_version': '6.2.2105-289-g53df468f0', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, '…

uh L2-error: 7.776E-03, uh conv rate: 1.52E-07
Lh L2-error: 5.714E-02, Lh conv rate: -7.39E-07
uh divErr: 0.0E+00
level: 2
Time to find EIG Val: 4.1E-02, MAX PREC LAM: 1.7E+00; MIN PREC LAM: 9.8E-01
IT:  1 cond: 1.70e+00 Assemble: 3.91e-02 Prec: 1.60e-02 Solve: 2.78e-02


...

uh L2-error: 7.776E-03, uh conv rate: -4.10E-14
Lh L2-error: 5.714E-02, Lh conv rate: -1.32E-11
uh divErr: 0.0E+00
level: 4
Time to find EIG Val: 4.1E-01, MAX PREC LAM: 1.8E+00; MIN PREC LAM: 9.8E-01
IT:  1 cond: 1.88e+00 Assemble: 9.47e-01 Prec: 1.77e-01 Solve: 9.98e-01


WebGuiWidget(value={'ngsolve_version': '6.2.2105-289-g53df468f0', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, '…

uh L2-error: 7.776E-03, uh conv rate: -2.56E-15
Lh L2-error: 5.714E-02, Lh conv rate: -5.59E-14
uh divErr: 0.0E+00
level: 5
Time to find EIG Val: 4.1E+00, MAX PREC LAM: 1.9E+00; MIN PREC LAM: 9.8E-01
IT:  1 cond: 1.90e+00 Assemble: 3.82e+00 Prec: 7.41e-01 Solve: 1.16e+01


WebGuiWidget(value={'ngsolve_version': '6.2.2105-289-g53df468f0', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, '…

uh L2-error: 7.776E-03, uh conv rate: 5.13E-15
Lh L2-error: 5.714E-02, Lh conv rate: 1.28E-14
uh divErr: 0.0E+00
821690
